<a href="https://colab.research.google.com/github/shubhangkhare/Learnings/blob/main/UP20%20Analytics%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Libraries

In [1]:
!pip install yfinance -q

In [2]:
!pip install fuzzywuzzy python-Levenshtein -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import requests
pd.set_option('display.max_columns', None)

# Plot Candlesticks

In [4]:
# Retrieve AAPL historical data
symbol = 'ADANIPOWER.NS' # Share symbol + NSE suffix
ticker = yf.Ticker(symbol)
data = ticker.history(period="1Y")

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data.index,
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])

# Customize the chart layout
fig.update_layout(title=f"{symbol} Candlestick Chart (1 Month)",
                  yaxis_title="Price",
                  xaxis_rangeslider_visible=False)

# Display the chart
fig.show()

# UP 20 - Code

## Reset index

In [5]:
data.reset_index(inplace = True)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-01-03 00:00:00+05:30,538.000000,544.500000,525.000000,544.500000,4044997,0.0,0.0
1,2024-01-04 00:00:00+05:30,548.000000,561.450012,537.299988,558.549988,3611727,0.0,0.0
2,2024-01-05 00:00:00+05:30,564.700012,565.000000,546.099976,551.099976,1411436,0.0,0.0
3,2024-01-08 00:00:00+05:30,553.950012,553.950012,540.000000,541.700012,975444,0.0,0.0
4,2024-01-09 00:00:00+05:30,545.000000,553.500000,535.000000,539.750000,1213712,0.0,0.0


## Get Candle Color

In [6]:
def green_candles(open, close):
    if close > open:
        return 'green'
    else:
        return 'red'

data['Candle Color'] = data.apply(lambda x: green_candles(x['Open'], x['Close']), axis = 1)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Candle Color
0,2024-01-03 00:00:00+05:30,538.000000,544.500000,525.000000,544.500000,4044997,0.0,0.0,green
1,2024-01-04 00:00:00+05:30,548.000000,561.450012,537.299988,558.549988,3611727,0.0,0.0,green
2,2024-01-05 00:00:00+05:30,564.700012,565.000000,546.099976,551.099976,1411436,0.0,0.0,red
3,2024-01-08 00:00:00+05:30,553.950012,553.950012,540.000000,541.700012,975444,0.0,0.0,red
4,2024-01-09 00:00:00+05:30,545.000000,553.500000,535.000000,539.750000,1213712,0.0,0.0,red


## Get UP 20 Dates

In [7]:
def check_up_20_helper(df_temp, threshold):
    """
    Check if the maximum price increase in a given DataFrame exceeds a specified threshold.

    Parameters:
    df_temp (pd.DataFrame): DataFrame containing stock market data with columns 'Date', 'High', 'Low', and 'Candle Color'.
    threshold (float): Percentage threshold to check against.

    Returns:
    pd.DataFrame or None: DataFrame with the start date, end date, and percentage increase if the condition is met, otherwise None.
    """

    color_pattern = tuple(df_temp['Candle Color'].values)

    if color_pattern[0] != 'green':
        return None

    if color_pattern == ('green', 'green', 'green'):
        pass
    elif color_pattern == ('green', 'green', 'red'):
        df_temp = df_temp.iloc[0:2, :]
    elif color_pattern in [('green', 'red', 'red'), ('green', 'red', 'green')]:
        df_temp = df_temp.iloc[0:1, :]  # Keep df_temp as a DataFrame with one row
    else:
        return None

    # Ensure df_temp is a DataFrame after slicing
    if isinstance(df_temp, pd.Series):
        df_temp = df_temp.to_frame().T

    # Calculate max and min prices
    max_price = df_temp['High'].max()
    min_price = df_temp['Low'].min()

    # Calculate percentage increase
    per_increase = 100 * (max_price - min_price) / min_price
    #print(per_increase)

    # Get start and end dates
    start_date = df_temp['Date'].iloc[0]  # Access the first element using .iloc[0]
    end_date = df_temp['Date'].iloc[-1]  # Access the last element using .iloc[-1]

    # Check if the percentage increase meets or exceeds the threshold
    if per_increase >= threshold:
        output_dict = {
            'Start Date': [start_date],
            'End Date': [end_date],
            '% Increase': [per_increase]
        }
        output_df = pd.DataFrame(output_dict)
        return output_df
    else:
        return None

#result = check_up_20_helper(df_temp, 5)
#result.head()

In [8]:
def check_up_20(symbol = "ADANIPOWER.NS", period="1mo", thres = 5):
    # Load Data
    ticker = yf.Ticker(symbol)
    data = ticker.history(period=period)
    # Reset Index
    data.reset_index(inplace = True)
    # Get Candle Color
    data['Candle Color'] = data.apply(lambda x: green_candles(x['Open'], x['Close']), axis = 1)

    result_list = []
    for idx in range(data.shape[0]):
        df_temp = data.iloc[idx:idx+3, :]
        #return df_temp
        output_df = check_up_20_helper(df_temp, thres)
        if output_df is not None:
            output_df['symbol'] = symbol
            result_list.append(output_df)
    if len(result_list) > 0:
        result = pd.concat(result_list)
        return result

In [9]:
# ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
result = check_up_20(symbol = "ADANIPOWER.NS", period="1Y", thres = 20)
result.head()

,Start Date,End Date,% Increase,symbol
0,2024-05-30 00:00:00+05:30,2024-06-03 00:00:00+05:30,32.610464,ADANIPOWER.NS
0,2024-05-31 00:00:00+05:30,2024-06-03 00:00:00+05:30,27.242382,ADANIPOWER.NS
0,2024-11-27 00:00:00+05:30,2024-11-28 00:00:00+05:30,35.091981,ADANIPOWER.NS


# Stock Screener

# Load UP 20 Companies

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df_screen = pd.read_excel('/content/drive/MyDrive/Algo Trading/UP20.xlsx')
df_screen.head()

,Name,Sub-Sector,Market Cap,Close Price,PE Ratio,Net Income,ROCE,Debt to Equity
0,Tata Consultancy Services Ltd,IT Services & Consulting,1500420.89,4147.00,32.68,45908.00,63.17,0.09
1,Infosys Ltd,IT Services & Consulting,757951.03,1829.95,28.89,26233.00,37.50,0.09
2,ITC Ltd,FMCG - Tobacco,598025.84,478.05,29.23,20458.78,34.87,0.00
3,Hindustan Unilever Ltd,FMCG - Household Products,589207.00,2507.70,57.33,10277.00,21.73,0.03
4,HCL Technologies Ltd,IT Services & Consulting,497251.98,1837.50,31.67,15702.00,28.21,0.08


## Get NSE symbols

In [12]:
# Define the API URL
url = "https://financialmodelingprep.com/api/v3/stock/list?apikey=urY2vTrf7Lw87kNKvWV5DKeg3x2eEhRU"
# Fetch data from the API
response = requests.get(url)
data = response.json()

# Convert JSON response to DataFrame
df = pd.DataFrame(data)

df = df[(df['exchangeShortName'] == 'NSE') & (df['type'] == 'stock')]
df['name'] = df['name'].str.replace('Limited', 'Ltd')

df.head()

,symbol,name,price,exchange,exchangeShortName,type
39479,SATINDLTD.NS,Sat Industries Ltd,114.85,National Stock Exchange of India,NSE,stock
39480,DHARMAJ.NS,Dharmaj Crop Guard Ltd,273.05,National Stock Exchange of India,NSE,stock
39484,VIPCLOTHNG.NS,VIP Clothing Ltd,46.79,National Stock Exchange of India,NSE,stock
39486,KEC.NS,KEC International Ltd,1223.80,National Stock Exchange of India,NSE,stock
39487,GMBREW.NS,G.M. Breweries Ltd,846.75,National Stock Exchange of India,NSE,stock


## Map NSE Symbols

In [13]:
from fuzzywuzzy import process
import pandas as pd

# Function to apply fuzzy matching
def get_best_match(query, choices, threshold=100):
    if query in choices:
        return query, 100
    match, score = process.extractOne(query, choices)
    return (match, score) if score >= threshold else (None, None)

# Create a list of names from the right DataFrame for fuzzy matching
name_choices = df['name'].tolist()

# Apply fuzzy matching to the 'Name' column in df_screen and create new columns with best matches and scores
df_screen[['fuzzy_match', 'Score']] = df_screen['Name'].apply(
    lambda x: pd.Series(get_best_match(x, name_choices))
)

# Remove Null Values
df = df[~df['name'].isna()]

# Now merge on the fuzzy matches
df_1 = pd.merge(df_screen, df, how='inner', left_on='fuzzy_match', right_on='name')

# Show the result
df_1.head()

,Name,Sub-Sector,Market Cap,Close Price,PE Ratio,Net Income,ROCE,Debt to Equity,fuzzy_match,Score,symbol,name,price,exchange,exchangeShortName,type
0,Tata Consultancy Services Ltd,IT Services & Consulting,1500420.89,4147.00,32.68,45908.00,63.17,0.09,Tata Consultancy Services Ltd,100.0,TCS.NS,Tata Consultancy Services Ltd,4099.90,National Stock Exchange of India,NSE,stock
1,Infosys Ltd,IT Services & Consulting,757951.03,1829.95,28.89,26233.00,37.50,0.09,Infosys Ltd,100.0,INFY.NS,Infosys Ltd,1938.75,National Stock Exchange of India,NSE,stock
2,ITC Ltd,FMCG - Tobacco,598025.84,478.05,29.23,20458.78,34.87,0.00,ITC Ltd,100.0,ITC.NS,ITC Ltd,481.60,National Stock Exchange of India,NSE,stock
3,Hindustan Unilever Ltd,FMCG - Household Products,589207.00,2507.70,57.33,10277.00,21.73,0.03,Hindustan Unilever Ltd,100.0,HINDUNILVR.NS,Hindustan Unilever Ltd,2406.25,National Stock Exchange of India,NSE,stock
4,HCL Technologies Ltd,IT Services & Consulting,497251.98,1837.50,31.67,15702.00,28.21,0.08,HCL Technologies Ltd,100.0,HCLTECH.NS,HCL Technologies Ltd,1946.65,National Stock Exchange of India,NSE,stock


In [14]:
#df_1 = pd.merge(df_screen, df, how='inner', left_on = 'Name', right_on = 'name')
#df_1.head()

In [15]:
from fuzzywuzzy import fuzz

# Calculate the fuzzy matching score between 'name' and 'symbol'
df_1['score_temp'] = df_1.apply(lambda row: fuzz.ratio(row['Name'], row['symbol']), axis=1)

# Group by 'name' and get the entry with the highest score
df_highest_score = df_1.loc[df_1.groupby('Name')['score_temp'].idxmax()]

# Reset index if necessary
df_highest_score.reset_index(drop=True, inplace=True)

df_1 = df_highest_score

In [16]:
df_1.shape, df_screen.shape

((241, 17), (281, 10))

## Find UP-20 Oppurtunities

In [17]:
def get_all(df, period="1mo", thres = 5):
    symbols = df['symbol'].to_list()
    result_list = []

    for symbol in symbols:
        result = check_up_20(symbol = symbol, period = period, thres = thres)
        if result is not None:
            result_list.append(result)

    if len(result_list) > 0:
        result = pd.concat(result_list)
        return result

In [18]:
df_all = get_all(df_1, period="1mo", thres = 18.5)
df_all.head()

ERROR:yfinance:$KENNAMET.NS: possibly delisted; no price data found  (period=1mo)
ERROR:yfinance:$KOVAI.NS: possibly delisted; no price data found  (period=1mo)


,Start Date,End Date,% Increase,symbol
0,2024-12-30 00:00:00+05:30,2024-12-31 00:00:00+05:30,20.296226,ASTRAZEN.NS
0,2024-12-04 00:00:00+05:30,2024-12-06 00:00:00+05:30,21.388440,BSE.NS
0,2024-12-05 00:00:00+05:30,2024-12-09 00:00:00+05:30,22.137650,BSE.NS
0,2024-12-16 00:00:00+05:30,2024-12-18 00:00:00+05:30,18.699187,BLUEJET.NS
0,2024-12-30 00:00:00+05:30,2025-01-01 00:00:00+05:30,25.837407,CRISIL.NS


In [19]:
df_all['symbol'].nunique()

14

In [20]:
df_all.sort_values(by = 'Start Date', inplace = True, ascending = False)
df_all.head(100)

,Start Date,End Date,% Increase,symbol
0,2025-01-01 00:00:00+05:30,2025-01-03 00:00:00+05:30,18.588309,LLOYDSME.NS
0,2024-12-31 00:00:00+05:30,2025-01-02 00:00:00+05:30,20.318065,TANLA.NS
0,2024-12-31 00:00:00+05:30,2025-01-01 00:00:00+05:30,19.576060,CRISIL.NS
0,2024-12-30 00:00:00+05:30,2024-12-31 00:00:00+05:30,20.296226,ASTRAZEN.NS
0,2024-12-30 00:00:00+05:30,2025-01-01 00:00:00+05:30,25.837407,CRISIL.NS
0,2024-12-18 00:00:00+05:30,2024-12-20 00:00:00+05:30,20.341882,KFINTECH.NS
0,2024-12-17 00:00:00+05:30,2024-12-19 00:00:00+05:30,20.013152,KSL.NS
0,2024-12-16 00:00:00+05:30,2024-12-18 00:00:00+05:30,24.482140,KSL.NS
0,2024-12-16 00:00:00+05:30,2024-12-18 00:00:00+05:30,18.699187,BLUEJET.NS
0,2024-12-16 00:00:00+05:30,2024-12-18 00:00:00+05:30,25.715900,CLSEL.NS
